In [1]:
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import PyPDF2
import os
import pickle

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Yatharth
[nltk_data]     Kohli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Yatharth
[nltk_data]     Kohli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
def preprocess(text):
    """Text cleaning and preprocessing."""
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    cleaned_tokens = [word.lower() for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(cleaned_tokens)

In [9]:
def classify_resume(file_path, model, vectorizer):
    """Classify uploaded resumes."""
    if file_path.endswith('.pdf'):
        with open(file_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            text = " ".join(page.extract_text() for page in reader.pages)
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    else:
        return "Unsupported file format. Use .txt or .pdf."
    cleaned_text = preprocess(text)
    text_vector = vectorizer.transform([cleaned_text])
    prediction = model.predict(text_vector)[0]
    return 'Flagged' if prediction == 1 else 'Not Flagged'

In [114]:
model_path = 'resume_classifier.pkl'

In [15]:
resume_df = pd.read_csv(r'C:\Users\Yatharth Kohli\OneDrive\Documents\Data Science lab\datasets\resume_dataset\Resume\Resume.csv')
resume_df.head()

ID                                         Resume_str  \
0  16852973           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1  22323967           HR SPECIALIST, US HR OPERATIONS      ...   
2  33176873           HR DIRECTOR       Summary      Over 2...   
3  27018550           HR SPECIALIST       Summary    Dedica...   
4  17812897           HR MANAGER         Skill Highlights  ...   

                                         Resume_html Category  
0  <div class="fontsize fontface vmargins hmargin...       HR  
1  <div class="fontsize fontface vmargins hmargin...       HR  
2  <div class="fontsize fontface vmargins hmargin...       HR  
3  <div class="fontsize fontface vmargins hmargin...       HR  
4  <div class="fontsize fontface vmargins hmargin...       HR

In [116]:
resume_df = pd.read_csv('resume.csv', encoding='latin-1')
resume_df['class'] = np.where(resume_df['class'] == 'flagged', 1, 0)
resume_df['cleaned'] = resume_df['resume_text'].apply(preprocess)

In [118]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(resume_df['cleaned'])
y = resume_df['class']

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [122]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [124]:
with open(model_path, 'wb') as f:
        pickle.dump((model, vectorizer), f)

In [104]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# ✅ Confusion matrix for the test data
y_test_pred = model.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred)

# ✅ Confusion matrix for the training data
y_train_pred = model.predict(X_train)
cm_train = confusion_matrix(y_train, y_train_pred)

# 🔥 Plotting the confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot for test data
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=["Not Flagged", "Flagged"])
disp_test.plot(cmap=plt.cm.Blues, ax=axes[0])
axes[0].set_title("Confusion Matrix - Test Data")

# Plot for training data
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=["Not Flagged", "Flagged"])
disp_train.plot(cmap=plt.cm.Greens, ax=axes[1])
axes[1].set_title("Confusion Matrix - Training Data")

plt.tight_layout()
plt.show()


In [126]:
y_pred = model.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.73      0.73        15
           1       0.60      0.60      0.60        10

    accuracy                           0.68        25
   macro avg       0.67      0.67      0.67        25
weighted avg       0.68      0.68      0.68        25



In [128]:
upload_dir = 'uploaded_resumes'
os.makedirs(upload_dir, exist_ok=True)


In [132]:
for file_name in os.listdir(upload_dir):
    file_path = os.path.join(upload_dir, file_name)
    result = classify_resume(file_path, model, vectorizer)
    print(f"{file_name}: {result}")

aadhar_card_yatharth.pdf: Flagged
yatharthkohli_resume.pdf: Flagged
